# BentoML Demo: Iris Classifier with custom web UI

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to __serve a Iris Classification model containing a REST API server with Custom Web UI__.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=h2o&ea=h2o-loan-default-prediction&dt=h2o-loan-default-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install bentoml
!pip install sklearn

## Create BentoService for model serving

In [3]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):

    @api(input=DataframeInput())
    def predict(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Writing iris_classifier.py


In [4]:
from sklearn import svm
from sklearn import datasets

from iris_classifier import IrisClassifier

if __name__ == "__main__":
  # Load training data
  iris = datasets.load_iris()
  X, y = iris.data, iris.target

  # Model Training
  clf = svm.SVC(gamma='scale')
  clf.fit(X, y)

  # Create a iris classifier service instance
  iris_classifier_service = IrisClassifier()

  # Pack the newly trained model artifact
  iris_classifier_service.pack('model', clf)

  # Save the prediction service to disk for model serving
  saved_path = iris_classifier_service.save()

Writing main.py


In [5]:
!python3 main.py

[2020-07-07 11:37:23,967] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be packaged together with saved bundle created, under './bundled_pip_dependencies' directory. For using a modified version of BentoML for production deployment, it is recommended to set the 'core/bentoml_deploy_version' config to a http location or your BentoML for on github, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-07-07 11:37:30,939] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manif

## Model Serving via REST API

Start local API model server with `bentoml serve` command:

In [6]:
!bentoml serve IrisClassifier:latest

[2020-07-07 11:37:33,682] INFO - Getting latest version IrisClassifier:20200707113724_204699
[2020-07-07 11:37:34,772] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be packaged together with saved bundle created, under './bundled_pip_dependencies' directory. For using a modified version of BentoML for production deployment, it is recommended to set the 'core/bentoml_deploy_version' config to a http location or your BentoML for on github, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-07-07 11:37:34,785] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1,  but loading from BentoML version 0.8.1+8.ge89f7fc
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jul/

At this point you can test out the Rest API server either by opening http://localhost:5000 in a new tab which will serve the swagger docs:

![alt text](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/swagger.png)


or by making a curl request in a another terminal window:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[1,2,1,2]]' \
localhost:5000/predict
```

## Adding Custom Web Static Content


In [7]:
!curl https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/static.tar.xz -o static.tar.xz
!tar --xz -xf static.tar.xz
!rm static.tar.xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k  100  197k    0     0  4481k      0 --:--:-- --:--:-- --:--:-- 4481k


Here we have a very simple web ui as our static content that bento will serve.
Now we will edit our bento service to point to this static directory.

Add `@web_static_content('./static')` to `iris_classifier.py`

**Note**: The path can be both relative or absolute. 

In [11]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService, web_static_content
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
@web_static_content('./static')
class IrisClassifier(BentoService):

    @api(input=DataframeInput())
    def predict(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


In [12]:
!python3 main.py

[2020-07-07 11:38:38,441] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be packaged together with saved bundle created, under './bundled_pip_dependencies' directory. For using a modified version of BentoML for production deployment, it is recommended to set the 'core/bentoml_deploy_version' config to a http location or your BentoML for on github, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-07-07 11:38:45,162] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manif

In [13]:
!bentoml serve IrisClassifier:latest

[2020-07-07 11:39:22,727] INFO - Getting latest version IrisClassifier:20200707113838_243E46
[2020-07-07 11:39:23,814] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be packaged together with saved bundle created, under './bundled_pip_dependencies' directory. For using a modified version of BentoML for production deployment, it is recommended to set the 'core/bentoml_deploy_version' config to a http location or your BentoML for on github, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-07-07 11:39:23,828] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1,  but loading from BentoML version 0.8.1+8.ge89f7fc
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jul/

Now if you visit http://localhost:5000/, you should be served with a beautiful UI:

![Custom UI](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/webui.png)

It's still possible to access the swagger docs at `/docs`